<a href="https://colab.research.google.com/github/heon-1219/David-Learning-Machine-Learning/blob/main/SimpleRNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This data will be used as input for the Simple RNN

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

#X train data
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(np.shape(train_X))

(1, 4, 5)


In [ ]:
rnn = SimpleRNN(3, return_sequences = True, return_state = True)
# rnn = SimpleRNN(3, return_sequences = False, return_state = False); implicit
# return sequences makes every hidden state visible.
# return_state returns the last hidden state
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))


hidden state : [[[-0.44785845 -0.7553059   0.98803157]
  [-0.99610585  0.5773123   0.98155165]
  [-0.99831593 -0.40003398  0.22512957]
  [-0.9827681  -0.01502109  0.93560565]]], shape: (1, 4, 3)
last hidden state : [[-0.9827681  -0.01502109  0.93560565]], shape: (1, 3)


In [ ]:
#LSTM Example

lstm = LSTM(3, return_sequences = False, return_state = True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

# unlike SimpleRNN, returns three values
# unlike SimpleRNN, ZLSTM returns cell state as well.

hidden state : [[-0.13369696 -0.10939941  0.5197757 ]], shape: (1, 3)
last hidden state : [[-0.13369696 -0.10939941  0.5197757 ]], shape: (1, 3)
last cell state : [[-0.37118372 -0.1331717   1.0408454 ]], shape: (1, 3)


In [ ]:
#Bidirectional(LSTM)

k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))


hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [ ]:
#Text Generation using RNN
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

text = """A Horse Is Racing \n
His Words Are The Law\n
One Ill Word Asks Another\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('size of the word set: %d' % vocab_size)

print('integer index: \n', tokenizer.word_index)

size of the word set: 15
integer index: 
 {'a': 1, 'horse': 2, 'is': 3, 'racing': 4, 'his': 5, 'words': 6, 'are': 7, 'the': 8, 'law': 9, 'one': 10, 'ill': 11, 'word': 12, 'asks': 13, 'another': 14}


In [ ]:
sequences = list()

for line in text.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('number of samples in learning process: %d' % len(sequences))
print('\n', sequences)

number of samples in learning process: 11

 [[1, 2], [1, 2, 3], [1, 2, 3, 4], [5, 6], [5, 6, 7], [5, 6, 7, 8], [5, 6, 7, 8, 9], [10, 11], [10, 11, 12], [10, 11, 12, 13], [10, 11, 12, 13, 14]]


In [ ]:
#embedding process

max_len = max(len(l) for l in sequences)
print('longest sample : {}'.format(max_len))
#longest sample #

#for the size of the longest sample #, start embedding
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
print('\n', sequences)

sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X)
print('\n', y)

longest sample : 5

 [[ 0  0  0  1  2]
 [ 0  0  1  2  3]
 [ 0  1  2  3  4]
 [ 0  0  0  5  6]
 [ 0  0  5  6  7]
 [ 0  5  6  7  8]
 [ 5  6  7  8  9]
 [ 0  0  0 10 11]
 [ 0  0 10 11 12]
 [ 0 10 11 12 13]
 [10 11 12 13 14]]
[[ 0  0  0  1]
 [ 0  0  1  2]
 [ 0  1  2  3]
 [ 0  0  0  5]
 [ 0  0  5  6]
 [ 0  5  6  7]
 [ 5  6  7  8]
 [ 0  0  0 10]
 [ 0  0 10 11]
 [ 0 10 11 12]
 [10 11 12 13]]

 [ 2  3  4  6  7  8  9 11 12 13 14]


In [ ]:
#label is now separated, one-hot encoding starts now

y = to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
#training RNN model starts now

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

#use softmax to categorize, cross entropy for loss and epoch = 200

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 400, verbose = 2)


Epoch 1/400
1/1 - 1s - loss: 2.6797 - accuracy: 0.0000e+00 - 1s/epoch - 1s/step
Epoch 2/400
1/1 - 0s - loss: 2.6706 - accuracy: 0.0909 - 18ms/epoch - 18ms/step
Epoch 3/400
1/1 - 0s - loss: 2.6614 - accuracy: 0.1818 - 13ms/epoch - 13ms/step
Epoch 4/400
1/1 - 0s - loss: 2.6519 - accuracy: 0.2727 - 9ms/epoch - 9ms/step
Epoch 5/400
1/1 - 0s - loss: 2.6423 - accuracy: 0.2727 - 8ms/epoch - 8ms/step
Epoch 6/400
1/1 - 0s - loss: 2.6324 - accuracy: 0.3636 - 7ms/epoch - 7ms/step
Epoch 7/400
1/1 - 0s - loss: 2.6223 - accuracy: 0.3636 - 8ms/epoch - 8ms/step
Epoch 8/400
1/1 - 0s - loss: 2.6119 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 9/400
1/1 - 0s - loss: 2.6012 - accuracy: 0.3636 - 12ms/epoch - 12ms/step
Epoch 10/400
1/1 - 0s - loss: 2.5901 - accuracy: 0.3636 - 8ms/epoch - 8ms/step
Epoch 11/400
1/1 - 0s - loss: 2.5788 - accuracy: 0.3636 - 8ms/epoch - 8ms/step
Epoch 12/400
1/1 - 0s - loss: 2.5670 - accuracy: 0.3636 - 9ms/epoch - 9ms/step
Epoch 13/400
1/1 - 0s - loss: 2.5549 - accuracy: 0.27

In [ ]:
#sentence generator for testing

def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    #iterate n times
    for _ in range(n):
      encoded = tokenizer.texts_to_sequences(current_word)[0]
      encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre')
      # for input X, predict Y, and save Y in result
      result = model.predict(encoded, verbose=0)
      result = np.argmax(result, axis=1)

      for word, index in tokenizer.word_index.items():
        #if the current index matches a word in results
        if index == result:
          break

      #add the word to current word
      current_word = current_word + ' ' + word
      #save the word to sentence
      sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
#print results

print(sentence_generation(model, tokenizer, 'The', 3))
print(sentence_generation(model, tokenizer, 'His', 2))
print(sentence_generation(model, tokenizer, 'words', 3))

The words words words
His words words
words words words words


In [ ]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')
sentences = []
for sentence in f:
  sentence = sentence.strip()
  sentence = sentence.lower()
  sentence = sentence.decode('ascii', 'ignore')

  if len(sentence) > 0:
    sentences.append(sentence)

f.close()

In [ ]:
print(sentences[:5])

["*** start of the project gutenberg ebook alice's adventures in", 'wonderland ***', '[illustration]', 'alices adventures in wonderland', 'by lewis carroll']


In [ ]:
sentences[:5]

["*** start of the project gutenberg ebook alice's adventures in",
 'wonderland ***',
 '[illustration]',
 'alices adventures in wonderland',
 'by lewis carroll']

In [ ]:
total_data = ''.join(sentences)
print('total length of the text: %d' % len(total_data))

print(total_data[:200])

total length of the text: 137826
*** start of the project gutenberg ebook alice's adventures inwonderland ***[illustration]alices adventures in wonderlandby lewis carrollthe millennium fulcrum edition 3.0contentschapter i.     down t


In [ ]:
char_vocab = sorted(list(set(total_data)))
vocab_size = len(char_vocab)
print('size of the set: {}'. format(vocab_size))

size of the set: 43


In [ ]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print('set :', char_to_index)

set : {' ': 0, '!': 1, "'": 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, '0': 9, '3': 10, ':': 11, ';': 12, '?': 13, '[': 14, ']': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42}


In [ ]:
index_to_char = {}
for key, value in char_to_index.items():
  index_to_char[value] = key

In [ ]:
train_X = 'appl'
train_y = 'pple'

In [ ]:
seq_length = 60

n_samples = int(np.floor((len(total_data)-1)/ seq_length))
print('sample #: {}'.format(n_samples))

sample #: 2297


In [ ]:
train_X = []
train_y = []

for i in range(n_samples):
  X_sample = total_data[i * seq_length: (i+1) * seq_length]

  X_encoded = [char_to_index[c] for c in X_sample]
  train_X.append(X_encoded)

  y_sample = total_data[i * seq_length + 1: (i+1) * seq_length + 1]
  y_encoded = [char_to_index[c] for c in y_sample]
  train_y.append(y_encoded)

print('X 1st sample: ', train_X[0])
print('y 1st sample: ', train_y[0])
print('-'*50)
print('X first sample decoding: ', [index_to_char[i] for i in train_X[0]])
print('y first sample decoding: ', [index_to_char[i] for i in train_y[0]])

X 1st sample:  [5, 5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0]
y 1st sample:  [5, 5, 0, 35, 36, 17, 34, 36, 0, 31, 22, 0, 36, 24, 21, 0, 32, 34, 31, 26, 21, 19, 36, 0, 23, 37, 36, 21, 30, 18, 21, 34, 23, 0, 21, 18, 31, 31, 27, 0, 17, 28, 25, 19, 21, 2, 35, 0, 17, 20, 38, 21, 30, 36, 37, 34, 21, 35, 0, 25]
--------------------------------------------------
X first sample decoding:  ['*', '*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'a', 'l', 'i', 'c', 'e', "'", 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ']
y first sample decoding:  ['*', '*', ' ', 's', 't', 'a', 'r', 't', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c'

In [ ]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_X.shape[2]),
return_sequences = True))
model.add(LSTM(hidden_units, return_sequences = True))
model.add(TimeDistributed(Dense(vocab_size, activation = 'softmax')))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.fit(train_X, train_y, epochs = 80, verbose = 2)

train_X의 크기(shape) : (2297, 60, 43)
train_y의 크기(shape) : (2297, 60, 43)
Epoch 1/80
72/72 - 45s - loss: 3.0557 - accuracy: 0.1717 - 45s/epoch - 621ms/step
Epoch 2/80
72/72 - 44s - loss: 2.7529 - accuracy: 0.2351 - 44s/epoch - 605ms/step
Epoch 3/80
72/72 - 47s - loss: 2.4237 - accuracy: 0.3195 - 47s/epoch - 655ms/step
Epoch 4/80
72/72 - 46s - loss: 2.2772 - accuracy: 0.3564 - 46s/epoch - 634ms/step
Epoch 5/80
72/72 - 36s - loss: 2.1738 - accuracy: 0.3845 - 36s/epoch - 502ms/step
Epoch 6/80
72/72 - 32s - loss: 2.0917 - accuracy: 0.4071 - 32s/epoch - 445ms/step
Epoch 7/80
72/72 - 34s - loss: 2.0137 - accuracy: 0.4258 - 34s/epoch - 478ms/step
Epoch 8/80
72/72 - 32s - loss: 1.9511 - accuracy: 0.4423 - 32s/epoch - 447ms/step
Epoch 9/80
72/72 - 33s - loss: 1.8967 - accuracy: 0.4552 - 33s/epoch - 463ms/step
Epoch 10/80
72/72 - 33s - loss: 1.8476 - accuracy: 0.4674 - 33s/epoch - 461ms/step
Epoch 11/80
72/72 - 33s - loss: 1.8039 - accuracy: 0.4797 - 33s/epoch - 452ms/step
Epoch 12/80
72/72 - 35s 

In [ ]:
def sentence_generation(model, length):
  ix = [np.random.randint(vocab_size)]

  y_char = [index_to_char[ix[-1]]]
  print(ix[-1], ' character', y_char[-1], 'is used to start prediction!')

  X = np.zeros((1, length, vocab_size))

  for i in range(length):

    X[0][i][ix[-1]] = 1
    print(index_to_char[ix[-1]], end = "")
    ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
    y_char.append(index_to_char[ix[-1]])

  return ('').join(y_char)

In [ ]:
result = sentence_generation(model, 100)
print(result)

9  character 0 is used to start prediction!
1/1 [==============================] - 0s 64ms/step
0comfortable for the dormouse, thought alice; only, as its aslee, as she spoke uphe sharply she said,


In [ ]:
#now that RNN is trained, actual text generation begins.

import numpy as np
from tensorflow.keras.utils import to_categorical

raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

tokens = raw_text.split()
raw_text = ' '.join(tokens)
print(raw_text)




I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [ ]:
#vocabulary creation

char_vocab = sorted(list(set(raw_text)))
vocab_size = len(char_vocab)
print('char set: ', char_vocab)
print('size of char set: {}'.format(vocab_size))

char set:  [' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']
size of char set: 33


In [ ]:
#integer indexing

char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [ ]:
length = 11
sequences = []
for i in range(length, len(raw_text)):
  seq = raw_text[i-length:i]
  sequences.append(seq)
print('total # of samples: %d' % len(sequences))

total # of samples: 426


In [ ]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [ ]:
sequences[30:45]

['mmer, I lik',
 'mer, I like',
 'er, I like ',
 'r, I like t',
 ', I like to',
 ' I like to ',
 'I like to c',
 ' like to co',
 'like to con',
 'ike to cont',
 'ke to conte',
 'e to contem',
 ' to contemp',
 'to contempl',
 'o contempla']

In [ ]:
encoded_sequences = []
for sequence in sequences: # getting 1 sample from the set each
  encoded_sequence = [char_to_index[char] for char in sequence] #integer indexing for each char
  encoded_sequences.append(encoded_sequence)

In [ ]:
encoded_sequences[:5]

[[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18],
 [0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28],
 [16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17],
 [14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0],
 [28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]]

In [ ]:
#separating X and y

encoded_sequences = np.array(encoded_sequences)

X_data = encoded_sequences[:, :-1]
y_data = encoded_sequences[:, -1]

print(X_data[:5])
print(y_data[:5])

[[ 8  0 16 14 28  0 24 23  0 31]
 [ 0 16 14 28  0 24 23  0 31 18]
 [16 14 28  0 24 23  0 31 18 28]
 [14 28  0 24 23  0 31 18 28 17]
 [28  0 24 23  0 31 18 28 17  0]]
[18 28 17  0 21]


In [ ]:
#one-hot encoding for bot X and y values

X_data_one_hot = [to_categorical(encoded, num_classes=vocab_size) for encoded in X_data]
X_data_one_hot = np.array(X_data_one_hot)
y_data_one_hot = to_categorical(y_data, num_classes=vocab_size)

print(X_data_one_hot.shape)

(426, 10, 33)


In [ ]:
#converting from one-hot to hidden, then to presentation level.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

hidden_units = 64

model = Sequential()
model.add(LSTM(hidden_units, input_shape = (X_data_one_hot.shape[1], X_data_one_hot.shape[2])))
model.add(Dense(vocab_size, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_data_one_hot, y_data_one_hot, epochs = 100, verbose = 2)


Epoch 1/100
14/14 - 3s - loss: 3.4728 - accuracy: 0.0915 - 3s/epoch - 225ms/step
Epoch 2/100
14/14 - 0s - loss: 3.3648 - accuracy: 0.1995 - 93ms/epoch - 7ms/step
Epoch 3/100
14/14 - 0s - loss: 3.0994 - accuracy: 0.1972 - 93ms/epoch - 7ms/step
Epoch 4/100
14/14 - 0s - loss: 3.0057 - accuracy: 0.1972 - 93ms/epoch - 7ms/step
Epoch 5/100
14/14 - 0s - loss: 2.9736 - accuracy: 0.1972 - 99ms/epoch - 7ms/step
Epoch 6/100
14/14 - 0s - loss: 2.9442 - accuracy: 0.1972 - 88ms/epoch - 6ms/step
Epoch 7/100
14/14 - 0s - loss: 2.9245 - accuracy: 0.1972 - 88ms/epoch - 6ms/step
Epoch 8/100
14/14 - 0s - loss: 2.9085 - accuracy: 0.1972 - 91ms/epoch - 6ms/step
Epoch 9/100
14/14 - 0s - loss: 2.8883 - accuracy: 0.1972 - 118ms/epoch - 8ms/step
Epoch 10/100
14/14 - 0s - loss: 2.8593 - accuracy: 0.1972 - 106ms/epoch - 8ms/step
Epoch 11/100
14/14 - 0s - loss: 2.8308 - accuracy: 0.2066 - 90ms/epoch - 6ms/step
Epoch 12/100
14/14 - 0s - loss: 2.7901 - accuracy: 0.1972 - 96ms/epoch - 7ms/step
Epoch 13/100
14/14 - 0s

In [ ]:
#sentence generation start

def sentence_generation(model, char_to_index, seq_length, seed_text, n):

  init_text = seed_text
  sentence = ''

  for _ in range(n):
    encoded = [char_to_index[char] for char in seed_text] #current sequence integer encoding
    encoded = pad_sequences([encoded], maxlen = seq_length, padding = 'pre') #padding
    encoded = to_categorical(encoded, num_classes = len(char_to_index)) #one-hot encoding

#for current X predict y and save to results

    result = model.predict(encoded, verbose = 0)
    result = np.argmax(result, axis = 1)

    for char, index in char_to_index.items():
      if index == result:
        break

    #current sequence + expected
    seed_text = seed_text + char
    #save to the current sentence
    sentence = sentence + char

  # when next n-times of prediction is done, return the sentence.
  sentence = init_text + sentence
  return sentence


In [ ]:
print(sentence_generation(model, char_to_index, 10, 'I get on w',80))

I get on with lofe aane mor  indg saade beorr  Bt wht oo lorsm min t aadng trm m ,inddgane
